In [21]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from collections import Counter
from glob import glob

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x7F7805252840

In [16]:
pardir = '../../data/value-data/'

In [17]:
mnli = pd.read_csv(pardir + 'MNLI/*.tsv', sep=',')
mnli = mnli[mnli['verified'].isin(['silver', 'gold'])]
mnli.shape

(715, 14)

In [19]:
mnli.loc[:, ['genre', 'sentence1', 'sentence2', 'gold_label']]

,genre,sentence1,sentence2,gold_label
961,government,"In-depth, very extensive case studies of several water catchment areas were conducted, and the final report is based on a synthesis of the findings from the case studies-another example of integration of findings across diverse sites (U.S.",Ain't nobody done ever taken no time to study water catchment areas.,contradiction
1268,government,An additional reporting standard for financial audits conducted in accordance with GAGAS,It is additional reporting standards for financial audits.,entailment
1824,fiction,He ripped the claw away and the fat man's shouting ceased.,The man continued to scream long after the claw was ripped away.,contradiction
2445,fiction,"You get us a working Franklin in two weeks, and we'll pretend everything is happening exactly as it should.","There was a deadline imposed to get Franklin working in two weeks, otherwise there would be an issue.",entailment
2720,telephone,really oh really i ain't know any of this ain't none of my friends done failed the English or the education ones,Every one I know has failed the English or education ones at least once.,contradiction
...,...,...,...,...
390883,government,It was obvious from the remarks made by a number of recently merged states some kind of critical phase done been passed.,This development is a very important landmark in the rollout of the program.,neutral
391719,telephone,but uh dental insurance is important to me it ain't as important as uh as health insurance you know if i had to rank them,"I'd say you should get health insurance first, but dental insurance has its uses.",entailment
391790,telephone,if if you want to go into one s really useful you're going to gotta go over a thousand dollars and even for a personal computer it's probably smart to spend that much and and we've ours ain't quite that powerful and so you know we are we are limited which may be one reason why,One should never spend more than $300 on a personal computer.,contradiction
392549,telephone,so Garland how nice is it now it's been a while since i've been there,How nice is it? I haven't been there in awhile,entailment


In [20]:
Counter(mnli['genre'])

Counter({'government': 130,
         'fiction': 83,
         'telephone': 160,
         'travel': 167,
         'slate': 175})

In [22]:
qnli = pd.read_csv(pardir + 'QNLI/train.tsv', sep=',')
qnli = qnli[qnli['verified'].isin(['silver', 'gold'])]
qnli.shape

(639, 6)

In [23]:
qnli

,Unnamed: 0,index,question,sentence,label,verified
124,124,124,Over how many people lived in Uruk?,It was little evidence of organized warfare or professional soldiers during the Uruk period and towns were generally unwalled.,not_entailment,gold
529,529,529,"Along with Germany, England, Spain and Australia, where has a research center on the history of emotions recently opened?","The history of emotions done become an increasingly popular topic recently, with some scholars arguing that it is an essential category of analysis, not unlike class, race, or gender.",not_entailment,silver
608,608,608,What cable network did the AFL sign a regional contract with?,"In some areas, such as with the Arizona Rattlers, Fox Sports affiliates still carry the games.",not_entailment,gold
619,619,619,"In the 2008 primary, how much of the Bronx vote did Obama get?","On the same day, John McCain been won 54.4% of the borough's 5,643 Republican votes, Mitt Romney 20.8%, Mike Huckabee 8.2%, Ron Paul 7.4%, Rudy Giuliani 5.6%, and the other candidates (Fred Thompson, Duncan Hunter and Alan Keyes) 3.6% between them",not_entailment,gold
742,742,742,What virtues should samurai not be dismissive of?,"""First of all, a samurai who dislikes battle and has not put his heart in the right place even though he has been born in the house of the warrior, should not be reckoned among one's retainers....",not_entailment,gold
...,...,...,...,...,...,...
103734,103675,103675,Stimulating nociceptors gon induce which type of pain?,Some nociceptors respond to more than one of these modalities and are consequently designated polymodal.,not_entailment,silver
104151,104092,104092,What is a common problem in antibiotic use?,"This done led to widespread problems with antimicrobial and antibiotic resistance, so much as to prompt the World Health Organization to classify antimicrobial resistance as a ""serious threat [ no longer a prediction for the future, it is happening right now in every region of the world and got the potential to affect anyone, of any age, in any country"".",entailment,silver
104407,104348,104348,Can illegal pinning methods result in being disqualified?,"Even if it ain't noticed, it's rare for such an attempt to result in a disqualification",entailment,gold
104661,104602,104602,Which ship was the target of Argentinian aggression?,"""it is a traitors in our midst"", wrote leader writer Ronald Spark on 7 May",not_entailment,gold


In [24]:
qqp = pd.read_csv(pardir + 'qqp/train.tsv', sep=',')
qqp = qqp[qqp['verified'].isin(['silver', 'gold'])]
qqp.shape

(2726, 8)

In [25]:
qqp

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,verified
108,108,187945,33176,32233,What are your new year resolutions for 2017?,what's,1,gold
314,314,341079,73803,7378,What are some characteristics of eccentric and concentric contractions?,What's some symptoms of eccentric and concentric contractions?,1,gold
492,492,33277,61163,61164,What are good metaphors in English?,What the best metaphors?,0,silver
521,521,328027,454530,454531,Who can win the US presidential elections?,Who you think will win this US presidential election?,1,gold
522,521,328027,454530,454531,Who can win the US presidential elections?,Who you think won this US presidential election?,1,gold
...,...,...,...,...,...,...,...,...
363136,363035,375154,14111,27618,How gon the ban of old 500 and 1000 rs notes help in bringing out the black money?,How does banning 500 and 1000 rupee notes help to control black money?,1,silver
363180,363079,128890,64271,207179,Which is the best laptop under Rs 25000?,What are some good laptop options below 25000?,1,gold
363361,363260,294279,23423,29379,What happens to our body after we die?,What happens if we die?,1,gold
363795,363694,372570,36709,41685,Which the best tourist place in Kerala?,What are the best places to visit on a 3-day trip in and around Kerala?,1,silver


In [26]:
rte = pd.read_csv(pardir + 'rte/train.tsv', sep=',')
rte = rte[rte['verified'].isin(['silver', 'gold'])]
rte.shape

(271, 6)

In [27]:
rte

,Unnamed: 0,index,sentence1,sentence2,label,verified
27,27,27,"When patients interrupt a course of antibiotics, the surviving bacteria return with a vengeance, often having rapidly mutated to resist the therapy.",Bacteria is winning the war against antibiotics.,not_entailment,silver
28,28,28,Initially the Bundesbank opposed the introduction of the euro but was compelled to accept it in light of the political pressure of the capitalist politicians who supported its introduction.,The introduction of the euro done been opposed.,entailment,silver
30,30,30,"One economic study will not be the basis of Canada's public policy decisions, but Easton's research does conclusively show that there are economic benefits in the legalization of marijuana.",Drug legalization got benefits.,entailment,silver
38,38,38,"The Christian Democrats (CDU) won 35.2% of the vote, or 225 seats, against 34.3% for Chancellor Gerhard Schroeder's Social Democrats (SPD).",It seem unlikely that dey gon be a coalition between Gerhard Schroeder Social Democrats and Angela Merkel Christian Democratic Union.,not_entailment,silver
41,41,41,Rockweed has been harvested commercially in Nova Scotia since the late 1950's and is currently the most important commercial seaweed in Atlantic Canada.,Marine vegetation harvested.,entailment,gold
...,...,...,...,...,...,...
2428,2421,2421,"Speaking of Jean Charles de Menezes, he was chased by armed officers into the station and shot five times at close range, his cousin, Alex Pereira, done hinted today that his family would sue Scotland Yard over the killing.",Jean Charles de Menezes is related to Alex Pereira.,entailment,gold
2430,2423,2423,"Women form half the population and 54% of the voters in the country, yet are very poorly represented in parliament.",Women are poorly represented in parliament.,entailment,silver
2435,2428,2428,"And this is a town that everybody agrees, law enforcement, Border Patrol, residents on both sides, is a staging area of illegal drugs and illegal aliens.",Steps are being taken to stop the smuggling of aliens.,not_entailment,silver
2448,2441,2441,"The campaign, which NSW Health has cost at AUD$600,000 will use a variety of print ads placed at bus stops and in youth magazines in addition to advertisements on websites such as MySpace and MSN. Print ads use a tag line saying ""Pot. It mightn't kill you, but it could turn you into a dickhead"".",People are likely to start smoking tobacco when young.,not_entailment,gold


In [29]:
sst2 = pd.read_csv(pardir + 'sst-2/train.tsv', sep=',')
sst2 = sst2[sst2['verified'].isin(['silver', 'gold'])]
sst2.shape

(140, 4)

In [30]:
sst2

,Unnamed: 0,sentence,label,verified
112,112,"takes a classic story , casts talented actors and uses a magnificent landscape to create a feature film",1,gold
113,112,"take a classic story , cast attractive and talented actors and use a magnificent landscape to create a feature film that be wickedly fun to watch",1,gold
450,449,"of the road , where the thematic ironies too obvious and the sexual politics too smug",0,silver
2392,2391,"we hate ( madonna ) within the film first five minutes , and she lack the skill or presence to regain any ground",0,silver
2486,2485,", still manage at least a decent attempt at meaningful cinema",1,silver
...,...,...,...,...
65469,65455,"good ,",1,silver
65579,65565,"drug abuse , infidelity and death are n't usually comedy no fare , but turpin film allow us to chuckle through the angst .",1,silver
65932,65918,"like mike is a slight and uninventive movie : like the exalted michael jordan referred to in the title , many can aspire but cann't none equal",0,silver
66882,66868,"for a movie about the power of poetry and passion , there is precious little of either .",0,gold


In [31]:
stsb = pd.read_csv(pardir + 'sts-b/train.tsv', sep=',')
stsb = stsb[stsb['verified'].isin(['silver', 'gold'])]
stsb.shape

(466, 12)

In [32]:
stsb

,Unnamed: 0,index,genre,filename,year,old_index,source1,source2,sentence1,sentence2,score,verified
1,1,1,main-captions,MSRvid,2012test,4,none,none,A man is playing a large flute.,A man is playing a flute.,3.8,silver
7,7,7,main-captions,MSRvid,2012test,13,none,none,The man is playing the piano.,The man playing guitar.,1.6,gold
11,11,11,main-captions,MSRvid,2012test,18,none,none,A woman picks up and holds a baby kangaroo.,A womans pickin up and holdin a baby kangaroo in her arms.,4.6,gold
12,11,11,main-captions,MSRvid,2012test,18,none,none,A woman picks up and holds a baby kangaroo.,A womans pickin up and holdin a baby kangaroo in her arms,4.6,gold
13,11,11,main-captions,MSRvid,2012test,18,none,none,A woman picks up and holds a baby kangaroo.,A woman picks up and holds a baby kangaroo in her arms.,4.6,gold
...,...,...,...,...,...,...,...,...,...,...,...,...
5569,5546,5546,main-news,headlines,2015,1486,none,none,World send emergency relief to battered Philippines,Typhoon Haiyan: world sends relief to battered Philippines,5.0,silver
5595,5572,5572,main-news,headlines,2016,164,Europe Media Monitor (http://emm.newsbrief.eu),Europe Media Monitor (http://emm.newsbrief.eu),Egypt braces for 'Friday of anger' after carnage,Egypt brace for Friday of anger after bloody crackdown,4.0,silver
5626,5603,5603,main-news,headlines,2016,420,Europe Media Monitor (http://emm.newsbrief.eu),Europe Media Monitor (http://emm.newsbrief.eu),Philippe becomes king of Belgium,Philippe ascend throne of divided Belgium,4.0,silver
5676,5653,5653,main-news,headlines,2016,839,Europe Media Monitor (http://emm.newsbrief.eu),Europe Media Monitor (http://emm.newsbrief.eu),US Attorney General Holder resign,US Attorney general Eric Holder to resign,4.0,silver


In [33]:
wnli = pd.read_csv(pardir + 'wnli/train.tsv', sep=',')
wnli = wnli[wnli['verified'].isin(['silver', 'gold'])]
wnli.shape

(268, 6)

In [34]:
wnli

,Unnamed: 0,index,sentence1,sentence2,label,verified
1,1,1,John couldn't see the stage with Billy in front of him because he is so short.,John so short.,1,silver
3,3,3,Steve follows Fred's example in everything. He influences him hugely.,Steve influences him hugely.,0,gold
5,5,5,"George got free tickets to the play, but he gave them to Eric, because he was particularly eager to see it.",George was particularly eager to see it.,0,silver
8,8,8,...but it simply ain't enough of them.,There were simply not enough copies of the newsletter.,1,gold
11,11,11,The table won't fit through the doorway because its too narrow.,The table is too narrow.,0,gold
...,...,...,...,...,...,...
642,616,616,"Emma's mother had died long ago, and her education had been managed by an excellent woman as governess.",Emma's mother's education had been managed by an excellent woman as governess.,0,gold
648,622,622,John couldn't see no stage w Billy in no front of him because he soo tall.,Billy is so tall.,1,silver
651,625,625,"Since it was raining, I carried da newspaper in my backpack to keep it dry.",I carried the newspaper in my backpack to keep the backpack dry.,0,silver
655,629,629,"Sam and Amy are passionately in love, but Amy's parents are unhappy about it, because they are fifteen.",Sam and Amy fifteen.,1,silver


In [35]:
davidson = pd.read_csv('../../data/davidson_hate.csv')
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [37]:
Counter(davidson['class'].values)

Counter({2: 4163, 1: 19190, 0: 1430})

class is the majority vote column. 2 means that it's neither hate speech nor offensive, 0 means hate speech and 1 means offensive

In [43]:
neodavid = davidson[davidson['class'].isin([0,2])].loc[:, ['tweet', 'class']].reset_index(drop=True)
neodavid['class'] = neodavid['class'].apply(lambda x: 1 if x==0 else 0)
neodavid = neodavid.rename(columns={'class': 'hate'})
neodavid

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...,0
1,""" momma said no pussy cats inside my doghouse """,0
2,"""@Addicted2Guys: -SimplyAddictedToGuys http://t.co/1jL4hi8ZMF"" woof woof hot scally lad",0
3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woof woof and hot soles",0
4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these dishes."" One oreo? Lol",0
...,...,...
5588,"you know what they say, the early bird gets the worm. *puts gummy worms in your morning coffee*",0
5589,you're all niggers,1
5590,you're such a retard i hope you get type 2 diabetes and die from a sugar rush you fucking faggot @Dare_ILK,1
5591,"you've gone and broke the wrong heart baby, and drove me redneck crazy",0


In [49]:
neodavid.to_csv('../../data/neodavid.tsv', index=False)